### Binary classification based on 3 layers neural network
#### author: Kim Jeong Min

##### load images

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.datasets
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pl

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import argparse
import sys
import os
import numpy as np
import time
import datetime 
import csv
import configparser
import argparse
import platform

from torch.autograd import Variable
from random import shuffle

import math

In [2]:
class Linear(nn.Module):

    def __init__(self, num_classes=2):

        super(Linear, self).__init__()

        self.ms = list()
        
        self.number_class   = num_classes

        _size_image     = 100* 100
        _num1           = 1024
        _num2           = 256
        _num3           = 64
        
        self.fc1        = nn.Linear(_size_image, _num1, bias=True)
        self.fc2        = nn.Linear(_num1, _num2, bias=True)
        self.fc3        = nn.Linear(_num2, _num3, bias=True)
        self.fc4        = nn.Linear(_num3, num_classes, bias=True)
        self.ms.append(self.fc1)
        self.ms.append(self.fc2)
        self.ms.append(self.fc3)
        self.ms.append(self.fc4)

        self.fc_layer1  = nn.Sequential(self.fc1, nn.ReLU())
        self.fc_layer2  = nn.Sequential(self.fc2, nn.ReLU())
        self.fc_layer3  = nn.Sequential(self.fc3, nn.ReLU())
        self.fc_layer4  = nn.Sequential(self.fc4, nn.Sigmoid())
        
        self.classifier = nn.Sequential(self.fc_layer1, self.fc_layer2, self.fc_layer3, self.fc_layer4)
        
        self._initialize_weight()        
        
    def _initialize_weight(self):

        for m in self.ms:
            
            n = m.in_features
            m.weight.data.uniform_(- 1.0 / math.sqrt(n), 1.0 / math.sqrt(n))

            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x):

        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x


In [3]:
transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])


#train_data_path = 'relative path of training data set'
train_data_path = 'horse-or-human/train'
trains_set = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
loader_train = torch.utils.data.DataLoader(trains_set, batch_size=64, shuffle=False, num_workers=1)  


test_data_path = 'horse-or-human/validation'
test_set = torchvision.datasets.ImageFolder(root=test_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
loader_test = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False, num_workers=1)  

num_classes = 2

##### load neural network model

In [4]:
model = Linear(num_classes=num_classes)

##### Set the flag for using cuda

In [5]:
bCuda = False
if bCuda:
    model.cuda()

##### optimization algorithm

In [6]:
LR = 0.01
optimizer   = optim.SGD(model.parameters(), lr=LR)
objective   = nn.CrossEntropyLoss()

##### function for training the model

In [7]:
def train():
    # print('train the model at given epoch')
    loss_train          = []
    acc_train           = []
    model.train()
    for idx_batch, (data, target) in enumerate(loader_train):
        if bCuda:
            data, target    = data.cuda(), target.cuda()
            
        data, target    = Variable(data), Variable(target)
        optimizer.zero_grad()
        
        output  = model(data)
        loss    = objective(output, target)

        loss.backward()
        optimizer.step()

        loss_train_batch    = loss.item() / len(data)
        loss_train.append(loss_train_batch)
        
        pred        = output.data.max(1)[1]
        correct     = pred.eq(target.data.view_as(pred)).cpu().sum()
        acc_train_batch   = 100. * float(correct) / len(data)
        acc_train.append(acc_train_batch)
        
    loss_train_mean     = np.mean(loss_train)
    loss_train_std      = np.std(loss_train)
    acc_train_mean      = np.mean(acc_train)
    acc_train_std       = np.std(acc_train)

    return {'loss_train_mean': loss_train_mean, 'loss_train_std': loss_train_std, 'acc_train_mean': acc_train_mean, 'acc_train_std': acc_train_std}

##### function for testing the model

In [8]:
def test():
    # print('test the model at given epoch')

    accuracy_test   = []
    loss_test       = 0
    correct         = 0

    model.eval()

    for idx_batch, (data, target) in enumerate(loader_test):

        if bCuda:
        
            data, target    = data.cuda(), target.cuda()

        data, target    = Variable(data), Variable(target)

        output  = model(data)
        loss    = objective(output, target)

        loss_test   += loss.item()
        pred        = output.data.max(1)[1]
        correct     += pred.eq(target.data.view_as(pred)).cpu().sum()

    loss_test       = loss_test / len(loader_test.dataset)
    accuracy_test   = 100. * float(correct) / len(loader_test.dataset)

    return {'loss_test': loss_test, 'accuracy_test': accuracy_test}

##### iteration for the epoch

In [9]:
epoch = 2000
loss_train_mean = list()
loss_train_std = list()
acc_train_mean = list()
acc_train_std = list()
loss_test = list()
accuracy_test = list()

for e in range(epoch):
        
    result_train    = train()
    result_test     = test()
    
    result_loss_train_mean = result_train['loss_train_mean']
    result_loss_train_std =  result_train['loss_train_std']
    result_acc_train_mean =  result_train['acc_train_mean']
    result_acc_train_std =  result_train['acc_train_std']
    result_loss_test =       result_test['loss_test']
    result_accuracy_test =   result_test['accuracy_test']

    loss_train_mean.append( result_loss_train_mean)
    loss_train_std.append(  result_loss_train_std)
    acc_train_mean.append(  result_acc_train_mean)
    acc_train_std.append(  result_acc_train_std)
    loss_test.append(       result_loss_test)
    accuracy_test.append(   result_accuracy_test)
    
    epoch_str = '[epoch '+ '{:05d}'.format(e)+ ']  '
    epoch_str += 'loss: '+ '(training)' + str(round(result_loss_train_mean, 5))+ ' (testing) ' + str(round(result_loss_test, 5)) + ', '
    epoch_str += 'accuracy: '+ '(training)' + str(round(result_acc_train_mean, 5))+ ' (testing) ' + str(round(result_accuracy_test, 5))
    print(epoch_str)
    
    #adaptive learning rate
    for g in optimizer.param_groups:
        g['lr'] = g['lr'] * 0.998

[epoch 00000]  loss: (training)0.02377 (testing) 0.01083, accuracy: (training)50.91912 (testing) 50.0
[epoch 00001]  loss: (training)0.0237 (testing) 0.01083, accuracy: (training)55.51471 (testing) 50.0
[epoch 00002]  loss: (training)0.02365 (testing) 0.01083, accuracy: (training)53.30882 (testing) 50.0
[epoch 00003]  loss: (training)0.02361 (testing) 0.01082, accuracy: (training)53.21691 (testing) 50.0
[epoch 00004]  loss: (training)0.02357 (testing) 0.01082, accuracy: (training)53.21691 (testing) 50.0
[epoch 00005]  loss: (training)0.02353 (testing) 0.01082, accuracy: (training)54.31985 (testing) 50.0
[epoch 00006]  loss: (training)0.02349 (testing) 0.01081, accuracy: (training)54.87132 (testing) 50.0
[epoch 00007]  loss: (training)0.02345 (testing) 0.01081, accuracy: (training)55.05515 (testing) 50.0
[epoch 00008]  loss: (training)0.02341 (testing) 0.01081, accuracy: (training)55.69853 (testing) 50.0
[epoch 00009]  loss: (training)0.02337 (testing) 0.01081, accuracy: (training)55.69

[epoch 00080]  loss: (training)0.0173 (testing) 0.01035, accuracy: (training)66.54412 (testing) 53.90625
[epoch 00081]  loss: (training)0.01717 (testing) 0.01034, accuracy: (training)66.36029 (testing) 54.29688
[epoch 00082]  loss: (training)0.01704 (testing) 0.01031, accuracy: (training)66.72794 (testing) 54.29688
[epoch 00083]  loss: (training)0.01691 (testing) 0.01026, accuracy: (training)67.1875 (testing) 55.07812
[epoch 00084]  loss: (training)0.01678 (testing) 0.01025, accuracy: (training)67.64706 (testing) 55.07812
[epoch 00085]  loss: (training)0.01667 (testing) 0.0102, accuracy: (training)68.19853 (testing) 55.46875
[epoch 00086]  loss: (training)0.01655 (testing) 0.01016, accuracy: (training)68.38235 (testing) 55.85938
[epoch 00087]  loss: (training)0.01643 (testing) 0.01014, accuracy: (training)68.84191 (testing) 56.25
[epoch 00088]  loss: (training)0.01632 (testing) 0.0101, accuracy: (training)69.02574 (testing) 56.25
[epoch 00089]  loss: (training)0.01621 (testing) 0.01006

[epoch 00158]  loss: (training)0.01336 (testing) 0.00774, accuracy: (training)83.82353 (testing) 81.25
[epoch 00159]  loss: (training)0.01334 (testing) 0.00773, accuracy: (training)83.91544 (testing) 81.25
[epoch 00160]  loss: (training)0.01333 (testing) 0.00772, accuracy: (training)84.00735 (testing) 81.25
[epoch 00161]  loss: (training)0.01331 (testing) 0.0077, accuracy: (training)84.00735 (testing) 81.25
[epoch 00162]  loss: (training)0.01329 (testing) 0.00769, accuracy: (training)84.00735 (testing) 81.25
[epoch 00163]  loss: (training)0.01327 (testing) 0.00767, accuracy: (training)84.09926 (testing) 81.25
[epoch 00164]  loss: (training)0.01325 (testing) 0.00766, accuracy: (training)84.09926 (testing) 81.64062
[epoch 00165]  loss: (training)0.01323 (testing) 0.00765, accuracy: (training)84.375 (testing) 81.64062
[epoch 00166]  loss: (training)0.01321 (testing) 0.00763, accuracy: (training)84.375 (testing) 82.03125
[epoch 00167]  loss: (training)0.01319 (testing) 0.00762, accuracy: (

[epoch 00235]  loss: (training)0.0125 (testing) 0.00723, accuracy: (training)90.44118 (testing) 83.98438
[epoch 00236]  loss: (training)0.0125 (testing) 0.00723, accuracy: (training)90.44118 (testing) 83.98438
[epoch 00237]  loss: (training)0.01249 (testing) 0.00723, accuracy: (training)90.44118 (testing) 84.375
[epoch 00238]  loss: (training)0.01249 (testing) 0.00723, accuracy: (training)90.44118 (testing) 84.375
[epoch 00239]  loss: (training)0.01248 (testing) 0.00723, accuracy: (training)90.625 (testing) 84.375
[epoch 00240]  loss: (training)0.01248 (testing) 0.00723, accuracy: (training)90.625 (testing) 84.375
[epoch 00241]  loss: (training)0.01247 (testing) 0.00722, accuracy: (training)90.71691 (testing) 84.375
[epoch 00242]  loss: (training)0.01247 (testing) 0.00722, accuracy: (training)90.71691 (testing) 83.98438
[epoch 00243]  loss: (training)0.01246 (testing) 0.00722, accuracy: (training)90.71691 (testing) 83.98438
[epoch 00244]  loss: (training)0.01246 (testing) 0.00722, accu

[epoch 00312]  loss: (training)0.01216 (testing) 0.00715, accuracy: (training)92.73897 (testing) 85.15625
[epoch 00313]  loss: (training)0.01215 (testing) 0.00715, accuracy: (training)92.73897 (testing) 85.15625
[epoch 00314]  loss: (training)0.01215 (testing) 0.00715, accuracy: (training)92.83088 (testing) 85.15625
[epoch 00315]  loss: (training)0.01214 (testing) 0.00715, accuracy: (training)92.83088 (testing) 85.15625
[epoch 00316]  loss: (training)0.01214 (testing) 0.00715, accuracy: (training)92.83088 (testing) 85.15625
[epoch 00317]  loss: (training)0.01214 (testing) 0.00715, accuracy: (training)92.83088 (testing) 85.15625
[epoch 00318]  loss: (training)0.01213 (testing) 0.00714, accuracy: (training)92.83088 (testing) 85.15625
[epoch 00319]  loss: (training)0.01213 (testing) 0.00714, accuracy: (training)92.92279 (testing) 85.15625
[epoch 00320]  loss: (training)0.01213 (testing) 0.00714, accuracy: (training)93.01471 (testing) 85.15625
[epoch 00321]  loss: (training)0.01212 (testin

[epoch 00389]  loss: (training)0.01189 (testing) 0.00694, accuracy: (training)94.20956 (testing) 86.32812
[epoch 00390]  loss: (training)0.01189 (testing) 0.00693, accuracy: (training)94.20956 (testing) 86.32812
[epoch 00391]  loss: (training)0.01188 (testing) 0.00693, accuracy: (training)94.20956 (testing) 86.32812
[epoch 00392]  loss: (training)0.01188 (testing) 0.00693, accuracy: (training)94.20956 (testing) 86.32812
[epoch 00393]  loss: (training)0.01188 (testing) 0.00692, accuracy: (training)94.20956 (testing) 86.32812
[epoch 00394]  loss: (training)0.01187 (testing) 0.00692, accuracy: (training)94.20956 (testing) 86.32812
[epoch 00395]  loss: (training)0.01187 (testing) 0.00692, accuracy: (training)94.20956 (testing) 86.32812
[epoch 00396]  loss: (training)0.01187 (testing) 0.00691, accuracy: (training)94.20956 (testing) 86.32812
[epoch 00397]  loss: (training)0.01186 (testing) 0.00691, accuracy: (training)94.30147 (testing) 86.32812
[epoch 00398]  loss: (training)0.01186 (testin

[epoch 00467]  loss: (training)0.01163 (testing) 0.00678, accuracy: (training)95.77206 (testing) 87.5
[epoch 00468]  loss: (training)0.01163 (testing) 0.00678, accuracy: (training)95.77206 (testing) 87.5
[epoch 00469]  loss: (training)0.01162 (testing) 0.00678, accuracy: (training)95.77206 (testing) 87.5
[epoch 00470]  loss: (training)0.01162 (testing) 0.00678, accuracy: (training)95.77206 (testing) 87.5
[epoch 00471]  loss: (training)0.01162 (testing) 0.00678, accuracy: (training)95.77206 (testing) 87.5
[epoch 00472]  loss: (training)0.01162 (testing) 0.00678, accuracy: (training)95.77206 (testing) 87.5
[epoch 00473]  loss: (training)0.01161 (testing) 0.00678, accuracy: (training)95.86397 (testing) 87.5
[epoch 00474]  loss: (training)0.01161 (testing) 0.00678, accuracy: (training)95.95588 (testing) 87.5
[epoch 00475]  loss: (training)0.01161 (testing) 0.00678, accuracy: (training)95.95588 (testing) 87.5
[epoch 00476]  loss: (training)0.0116 (testing) 0.00678, accuracy: (training)96.04

[epoch 00545]  loss: (training)0.01141 (testing) 0.00676, accuracy: (training)97.05882 (testing) 87.89062
[epoch 00546]  loss: (training)0.01141 (testing) 0.00676, accuracy: (training)97.05882 (testing) 87.5
[epoch 00547]  loss: (training)0.0114 (testing) 0.00676, accuracy: (training)97.05882 (testing) 87.5
[epoch 00548]  loss: (training)0.0114 (testing) 0.00676, accuracy: (training)97.05882 (testing) 87.5
[epoch 00549]  loss: (training)0.0114 (testing) 0.00676, accuracy: (training)97.05882 (testing) 87.5
[epoch 00550]  loss: (training)0.0114 (testing) 0.00676, accuracy: (training)97.15074 (testing) 87.5
[epoch 00551]  loss: (training)0.01139 (testing) 0.00676, accuracy: (training)97.15074 (testing) 87.5
[epoch 00552]  loss: (training)0.01139 (testing) 0.00676, accuracy: (training)97.15074 (testing) 87.5
[epoch 00553]  loss: (training)0.01139 (testing) 0.00675, accuracy: (training)97.15074 (testing) 87.5
[epoch 00554]  loss: (training)0.01139 (testing) 0.00675, accuracy: (training)97.1

[epoch 00623]  loss: (training)0.01125 (testing) 0.00689, accuracy: (training)97.70221 (testing) 86.71875
[epoch 00624]  loss: (training)0.01125 (testing) 0.0069, accuracy: (training)97.70221 (testing) 86.32812
[epoch 00625]  loss: (training)0.01125 (testing) 0.0069, accuracy: (training)97.70221 (testing) 86.32812
[epoch 00626]  loss: (training)0.01125 (testing) 0.0069, accuracy: (training)97.70221 (testing) 86.32812
[epoch 00627]  loss: (training)0.01124 (testing) 0.00691, accuracy: (training)97.70221 (testing) 86.32812
[epoch 00628]  loss: (training)0.01124 (testing) 0.00691, accuracy: (training)97.70221 (testing) 86.32812
[epoch 00629]  loss: (training)0.01124 (testing) 0.00691, accuracy: (training)97.70221 (testing) 86.32812
[epoch 00630]  loss: (training)0.01124 (testing) 0.00692, accuracy: (training)97.70221 (testing) 86.32812
[epoch 00631]  loss: (training)0.01124 (testing) 0.00692, accuracy: (training)97.70221 (testing) 86.32812
[epoch 00632]  loss: (training)0.01124 (testing) 

KeyboardInterrupt: 

##### plot result

In [ ]:
def print_history_graph(loss_history, loss_train_std, acc_history, acc_train_std, vloss_history, vacc_history):
    #plt.plot(acc_history, color='#ff0000', label='Train Accuracy')
    plt.errorbar(list(range(len(acc_history), acc_history, yerr=acc_train_std, ecolor = '#ffcccc', color='#ff0000', label='Train Loss')
    plt.plot(vacc_history, color='#0000ff', label='Validation Accuracy')
    plt.legend(['Train Accuracy','Validation Accuracy'])
    plt.title('Accuracy')
    plt.show()

    #plt.plot(loss_history, color='#ff0000', label='Train Loss')
    plt.errorbar(list(range(len(loss_history), loss_history, yerr=loss_train_std, ecolor = '#ffcccc', color='#ff0000', label='Train Loss')
    plt.plot(vloss_history, color='#0000ff', label='Validation Loss')
    plt.legend(['Train Loss','Validation Loss'])
    plt.title('Loss')
    plt.show()
    
    nx, ny = 3, 3
    data = (('dataset', 'loss', 'accuracy'), ('train', str(np.round(loss_history[-1],4)), str(np.round(acc_history[-1],4))), ('validation', str(np.round(vloss_history[-1],4)), str(np.round(vacc_history[-1],4))))
    pl.figure()
    tb = pl.table(cellText=data, loc=(0,0), cellLoc='center')

    tc = tb.properties()['child_artists']
    for cell in tc: 
        cell.set_height(1/ny)
        cell.set_width(1/nx)

    ax = pl.gca()
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
print_history_graph(loss_train_mean, loss_train_std, acc_train_mean, acc_train_std, loss_test, accuracy_test)